In [ ]:
# Özellik Çıkarımı ( Feature Extraction )
# Ham veriden değişken üretmek

In [1]:
# 1 - Binary Features: Flag, Bool, True-False 

In [2]:
import pandas as pd

def load():
    data = pd.read_csv("titanic.csv")
    return data

df = load()

In [3]:
# Dolu mu diye sorup çıkan değerleri 1 ve 0 ile yeni değişkene atıyor
df["NEW_CABIN_BOOL"] = df["Cabin"].notnull().astype("int")

In [5]:
# Kabin numarası olanlar daha çok hayatta kalmış
df.groupby("NEW_CABIN_BOOL").agg({"Survived":"mean"})

,Survived
NEW_CABIN_BOOL,
0,0.299854
1,0.666667


In [7]:
from statsmodels.stats.proportion import proportions_ztest

test_stat, pvalue = proportions_ztest(count=[df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].sum(),
                                             df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].sum()],
                                      
                                      nobs= [df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].shape[0],
                                             df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].shape[0]])
print("Test stat = % .4f, p-value = %.4f" % (test_stat,pvalue))

# p-value 0.5'ten küçük yani kabin numarası olanlar ile olmayanlar arasında fark var.

Test stat =  9.4597, p-value = 0.0000


In [8]:
# İki değişkeni toplayarak yeni bir değişkene atadık ve hayatta kalma oranlarına baktık.
df.loc[((df["SibSp"] + df["Parch"])>0), "NEW_IS_ALONE"] = "NO"
df.loc[((df["SibSp"] + df["Parch"]) == 0), "NEW_IS_ALONE"] = "YES"

df.groupby("NEW_IS_ALONE").agg({"Survived":"mean"})

,Survived
NEW_IS_ALONE,
NO,0.505650
YES,0.303538


In [ ]:
# ---------------------------------------------------------------- #

In [10]:
# 2 - Text Features -> Text'ler üzerinden özellik türetmek
df = load()

In [12]:
# Letter Count -> Harf
df["NEW_NAME_COUNT"] = df["Name"].str.len()

In [15]:
# Word Count -> Kelime
df["NEW_NAME_WORD_COUNT"] = df["Name"].apply(lambda x: len(str(x).split(" ")))
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_NAME_COUNT,NEW_NAME_WORD_COUNT
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,23,4
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,51,7
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,22,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,44,7
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,24,4


In [16]:
# Özel Yapıları Yakalamak
df["NEW_NAME_DR"] = df["Name"].apply(lambda x: len([x for x in x.split() if x.startswith("Dr")]))

In [22]:
# Doktor olanların hayatta kalma oranı
df.groupby("NEW_NAME_DR").agg({"Survived":["mean","count"]})

Survived      
                mean count
NEW_NAME_DR               
0            0.38252   881
1            0.50000    10

In [23]:
# ---------------------------------------------------------------- #

In [24]:
# 3 - Regex ile Değişken Türetmek

In [25]:
df = load()

In [27]:
df["NEW_TITLE"] = df.Name.str.extract(" ([A-Za-z]+)\.",expand=False)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_TITLE
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [28]:
df[["NEW_TITLE","Survived","Age"]].groupby(["NEW_TITLE"]).agg({"Survived":"mean","Age":["count","mean"]})

Survived   Age           
               mean count       mean
NEW_TITLE                           
Capt       0.000000     1  70.000000
Col        0.500000     2  58.000000
Countess   1.000000     1  33.000000
Don        0.000000     1  40.000000
Dr         0.428571     6  42.000000
Jonkheer   0.000000     1  38.000000
Lady       1.000000     1  48.000000
Major      0.500000     2  48.500000
Master     0.575000    36   4.574167
Miss       0.697802   146  21.773973
Mlle       1.000000     2  24.000000
Mme        1.000000     1  24.000000
Mr         0.156673   398  32.368090
Mrs        0.792000   108  35.898148
Ms         1.000000     1  28.000000
Rev        0.000000     6  43.166667
Sir        1.000000     1  49.000000

In [29]:
# ---------------------------------------------------------------- #

In [39]:
# Date Değişkenleri Üretmek
from datetime import date

dff = pd.read_csv("course_reviews.csv")
dff.head()
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rating              4323 non-null   float64
 1   Timestamp           4323 non-null   object 
 2   Enrolled            4323 non-null   object 
 3   Progress            4323 non-null   float64
 4   Questions Asked     4323 non-null   float64
 5   Questions Answered  4323 non-null   float64
dtypes: float64(4), object(2)
memory usage: 202.8+ KB


In [40]:
dff["Timestamp"] = pd.to_datetime(dff["Timestamp"], format="%Y-%m-%d")

In [41]:
# Yıl
dff["year"] = dff["Timestamp"].dt.year
# Ay
dff["month"] = dff["Timestamp"].dt.month

In [46]:
# Yıl farkı
dff["year_diff"] = date.today().year - dff["Timestamp"].dt.year

In [57]:
# month diff > iki tarih arasındaki ay farkı -> yıl farkı + ay farkı
dff["month_diff"] = (date.today().year - dff["Timestamp"].dt.year) * 12 + date.today().month - dff["Timestamp"].dt.month


In [60]:
dff["day_name"] = dff["Timestamp"].dt.day_name()

In [ ]:
# ---------------------------------------------------------------- #

In [62]:
# 4 - Özellik Etkileşimleri ( Feature Interaction )

In [64]:
df = load()

In [67]:
# Hangi yaşta hangi koltukta uçuyor
df["NEW_AGE_PCLASS"] = df["Age"] * df["Pclass"]

In [69]:
# Akrabalık sayıları ile ilgili iki değişkenin toplamı + kişinin kendisi -> Aile boyutu
df["NEW_FAMILY_SIZE"] = df["SibSp"] * df["Parch"] + 1

In [82]:
# Yaşı 21 den küçük erkekleri bul -> youngmale yaz ve yeni değişkene at
df.loc[(df["Sex"]=="male") & (df["Age"] <= 21), "NEW_SEX_CAT"] = "youngmale"

# Ortada kalan kısıma mature male dedik.
df.loc[(df["Sex"]=="male") & ((df["Age"] > 21) & (df["Age"]) <= 50), "NEW_SEX_CAT"] = "maturemale"

# Daha büyükleri
df.loc[(df["Sex"]=="male") & (df["Age"] > 50), "NEW_SEX_CAT"] = "seniormale"

# Yaşı 21 den küçük kadın bul -> youngmale yaz ve yeni değişkene at
df.loc[(df["Sex"]=="female") & (df["Age"] <= 21), "NEW_SEX_CAT"] = "youngfemale"

# Ortada kalan kısıma mature male dedik.
df.loc[(df["Sex"]=="female") & ((df["Age"] > 21) & (df["Age"]) <= 50), "NEW_SEX_CAT"] = "maturefemale"

# Daha büyükleri
df.loc[(df["Sex"]=="female") & (df["Age"] > 50), "NEW_SEX_CAT"] = "seniorfemale"

In [84]:
df.groupby("NEW_SEX_CAT")["Survived"].mean()

NEW_SEX_CAT
maturefemale    0.730640
maturemale      0.194340
seniorfemale    0.941176
seniormale      0.127660
Name: Survived, dtype: float64